This si to be ran in Lyon for exploring the parameter space: lmin, delta_ell, covcut

In [ ]:
from pylab import *
import os
import sys
import time
import pickle
from importlib import reload


# Specific science modules
import healpy as hp
import numpy as np
import scipy

# Specific qubic modules
from qubicpack.utilities import Qubic_DataDir
from pysimulators import FitsArray
#from mpi4py import MPI
# from pyoperators import MPI
import pysm
import qubic
from qubic import QubicSkySim as qss
from qubic import fibtools as ft
from qubic import camb_interface as qc
from qubic import SpectroImLib as si
from qubic import NamasterLib as nam
from qubic import mcmc
from qubic import AnalysisMC as amc

rc('figure', figsize=(16, 10))
rc('font', size=15)
plt.rcParams['image.cmap'] = 'jet'


In [ ]:
# ### Decode arguments
# outname = str(sys.argv[1])
# file_noise_profile = str(sys.argv[2])
# cov_file = str(sys.argv[3])
# clnoise_file = str(sys.argv[4])
# nbmc = int(sys.argv[5])
# signoise = float(sys.argv[6])
# lmin = int(sys.argv[7])
# delta_ell = int(sys.argv[8])
# covcut = float(sys.argv[9])
# is_spatial_corr = int(sys.argv[10])


### Decode arguments
outname = None

nbmc = 10

lmin = 40
delta_ell = 30
covcut = 0.1
is_spatial_corr = 1


In [ ]:
### Some initializations
### Initialize
global_dir = Qubic_DataDir(datafile='instrument.py', datadir=os.environ['QUBIC_DATADIR'])

### This is for the FI
#####eff_dur = 4
old_config = False
# configs = ['FI150', 'FI150'] #FI220 or FI150
# configs = ['FI220', 'FI220'] #FI220 or FI150
configs  = ['FI150', 'FI220'] #FI220 or FI150

# old_config=True
# configs = ['FI-150', 'FI-150'] #FI220 or FI150
#####eff_dur = 30

### Read some stuff
# Read dictionary chosen
dictfilename = global_dir + '/dicts/pipeline_demo.dict'
d = qubic.qubicdict.qubicDict()
d.read_from_file(dictfilename)
d['nside'] = 256
center = qubic.equ2gal(d['RA_center'], d['DEC_center'])
print(center)
nbands = 1
d['nf_recon'] = nbands
d['nf_sub'] = nbands
print('sub',d['nf_sub'])
print(d['effective_duration'])
####d['effective_duration'] = eff_dur
####print(d['effective_duration'])

# Frequency band
dA = d.copy()
dA['filter_nu'] = int(configs[0][-3:]) * 1e9
print(dA['filter_nu']/1e9)

dB = d.copy()
dB['filter_nu'] = int(configs[1][-3:]) * 1e9
print(dB['filter_nu']/1e9)



In [ ]:
## Make a sky using PYSM: It will have the expected QUBIC beam, the coverage and noise according to this coverage
## This creates a realization of the sky (a realization of the CMB is there is CMB in sly_config) 
seed = np.random.randint(1)
sky_config = {'cmb': seed}


Qubic_sky_A = qss.Qubic_sky(sky_config, dA)
Qubic_sky_B = qss.Qubic_sky(sky_config, dB)

##### Getting FastSimulator output maps
maps_qubicA, maps_noiselessA, maps_noiseA, coverageA = Qubic_sky_A.get_partial_sky_maps_withnoise(spatial_noise=False, 
                                                                                                  old_config=old_config,
                                                                                                 Nyears=dA['effective_duration'])
maps_qubicB, maps_noiselessB, maps_noiseB, coverageB = Qubic_sky_B.get_partial_sky_maps_withnoise(spatial_noise=False, 
                                                                                                  old_config=old_config,
                                                                                                 Nyears=dB['effective_duration'])


In [ ]:
################################# Displaying Results ##########################################
### Coverage map
hp.mollview(coverageA, title='Coverage Map '+configs[0], sub=(2,2,1))
hp.gnomview(coverageA, reso=25, title='Coverage Map '+configs[0], sub=(2,2,2))
hp.mollview(coverageB, title='Coverage Map '+configs[1], sub=(2,2,3))
hp.gnomview(coverageB, reso=25, title='Coverage Map '+configs[1], sub=(2,2,4))

figure()
ok = coverageA != 0
plot(coverageA[ok], coverageB[ok],',')
plot(linspace(0,1,100), linspace(0,1,100))

### Maps per sub-bands
stn = ['I', 'Q', 'U']
figure()
for s in range(3):
    hp.gnomview(maps_noiselessA[0,:,s], reso=25, sub=(2,3,s+1), 
                title='Noiseless '+stn[s]+' - '+configs[0] + ' - '+str(dA['effective_duration'])+'y')
    hp.gnomview(maps_noiselessB[0,:,s], reso=25, sub=(2,3,3+s+1), 
                title='Noiseless '+stn[s]+' - '+configs[1] + ' - '+str(dB['effective_duration'])+'y')
        
figure()
for s in range(3):
    hp.gnomview(maps_qubicA[0,:,s], reso=25, sub=(2,3,s+1), 
                title='QUBIC '+stn[s]+' - '+configs[0] + ' - '+str(dA['effective_duration'])+'y')
    hp.gnomview(maps_qubicB[0,:,s], reso=25, sub=(2,3,3+s+1), 
                title='QUBIC '+stn[s]+' - '+configs[1] + ' - '+str(dB['effective_duration'])+'y')
figure()   
for s in range(3):
    hp.gnomview(maps_noiseA[0,:,s], reso=25, sub=(2,3,s+1), 
                title='Noise '+stn[s]+' - '+configs[0] + ' - '+str(dA['effective_duration'])+'y')
    hp.gnomview(maps_noiseB[0,:,s], reso=25, sub=(2,3,3+s+1), 
                title='Noise '+stn[s]+' - '+configs[1] + ' - '+str(dB['effective_duration'])+'y')



In [ ]:
reload(qss)
rc('figure', figsize=(12, 10))

subplot(2,1,1)
xx, yyI, yyQ, yyU = qss.get_angular_profile(maps_noiseA[0,:,:], nbins=30, separate=True, center=[0,0])
pix_size = hp.nside2resol(256, arcmin=True)
plot(xx, yyI * pix_size, 'o',label='I')
plot(xx, yyQ * pix_size , 'o',label='Q')
plot(xx, yyU * pix_size, 'o',label='U')
meanvalI = np.mean(yyI[xx<10]) * pix_size
meanvalQU = np.mean((yyQ[xx<10]+yyQ[xx<10])/2) * pix_size
axhline(y=meanvalI, label=r'I RMS = {0:5.1f} $\mu K.arcmin$'.format(meanvalI), color='r', ls=':')
axhline(y=meanvalQU, label=r'QU RMS = {0:5.1f} $\mu K.arcmin$'.format(meanvalQU), color='m', ls=':')
xlim(0,20)
xlabel('Degrees from center of the field')
ylabel(r'Noise RMS $[\mu K.arcmin]$')
legend(fontsize=11)
ylim(0,meanvalQU*2)
title(configs[0] + ' - '+str(dA['effective_duration'])+'y')

subplot(2,1,2)
xx, yyI, yyQ, yyU = qss.get_angular_profile(maps_noiseB[0,:,:], nbins=30, separate=True, center=[0,0])
pix_size = hp.nside2resol(256, arcmin=True)
plot(xx, yyI * pix_size, 'o',label='I')
plot(xx, yyQ * pix_size , 'o',label='Q')
plot(xx, yyU * pix_size, 'o',label='U')
meanvalI = np.mean(yyI[xx<10]) * pix_size
meanvalQU = np.mean((yyQ[xx<10]+yyQ[xx<10])/2) * pix_size
axhline(y=meanvalI, label=r'I RMS = {0:5.1f} $\mu K.arcmin$'.format(meanvalI), color='r', ls=':')
axhline(y=meanvalQU, label=r'QU RMS = {0:5.1f} $\mu K.arcmin$'.format(meanvalQU), color='m', ls=':')
xlim(0,20)
xlabel('Degrees from center of the field')
ylabel(r'Noise RMS $[\mu K.arcmin]$')
legend(fontsize=11)
ylim(0,meanvalQU*2)
title(configs[1] + ' - '+str(dB['effective_duration'])+'y')

tight_layout()


In [ ]:
################################### Flat Weighting #################################################################
### Create a Namaster object
cov = coverageA.copy()
lmax = 2 * d['nside'] - 1
okpix = cov > np.max(cov) * covcut

### We use Flat weighting
maskpix = np.zeros(12*d['nside']**2)
maskpix[okpix] = 1
Namaster = nam.Namaster(maskpix, lmin=lmin, lmax=lmax, delta_ell=delta_ell)

In [ ]:
#######################################################################################################
#### Now in a function to loop over ell binning, lmin, coverage
def run_mc(nbmc, Namaster, cov, d, configs, verbose=False, clnoise=None, duration=4):
    
    #### Dictionnary for 150 GHz
    dA = d.copy()
    dA['effective_duration'] = duration
    dA['nside'] = 256
    dA['nf_sub']=1
    dA['filter_nu'] = int(configs[0][-3:]) * 1e9
    print(configs[0], dA['filter_nu']/1e9, dA['effective_duration'],'Years')
    
    dB = d.copy()
    dB['nside'] = 256
    dB['effective_duration'] = duration
    dB['nf_sub']=1
    dB['filter_nu'] = int(configs[1][-3:]) * 1e9
    print(configs[1], dB['filter_nu']/1e9, dB['effective_duration'],'Years')    
    
    ell_bins, b = Namaster.get_binning(d['nside'])
    mask_apo = Namaster.mask_apo
    okpix = cov > (np.max(cov)*0.1)


    seed = np.random.randint(1,100000)
    sky_config = {'cmb': seed}
    Qubic_sky_A = qss.Qubic_sky(sky_config, dA)
    Qubic_sky_B = qss.Qubic_sky(sky_config, dB)

    w=None
    cl_noise_qubic = np.zeros((nbmc, 1, len(ell_bins), 4))
    print('        Starting MC')
    for imc in range(nbmc):
        t0 = time.time()
        qubicnoiseA = Qubic_sky_A.get_partial_sky_maps_withnoise(spatial_noise=True, 
                                                                 noise_only=True, 
                                                                 Nyears=dA['effective_duration'],
                                                                 old_config=old_config)[0][0,:,:]
        qubicnoiseB = Qubic_sky_B.get_partial_sky_maps_withnoise(spatial_noise=True, 
                                                                 noise_only=True, 
                                                                 Nyears=dB['effective_duration'],
                                                                 old_config=old_config)[0][0,:,:]
        print(qubicnoiseA.shape)
        
        ### Compute Spectra:
        # Noise Only
        if verbose: print('   - QUBIC Noise maps')
        leff, cl_noise_qubic[imc, 0, :,:], w = Namaster.get_spectra(qubicnoiseA.T, 
                                                                 map2 = qubicnoiseB.T,
                                                                 purify_e=True, 
                                                                 purify_b=False, 
                                                                 w=w, 
                                                                 verbose=False,
                                                                 beam_correction=True,
                                                                 pixwin_correction=True)
        t1 = time.time()
        print('             Monte-Carlo: Iteration {0:} over {1:} done in {2:5.2f} sec'.format(imc, nbmc,t1-t0))
        
    
    # average MC results
    mcl_noise_qubic = np.mean(cl_noise_qubic, axis=0)[0]
    scl_noise_qubic = np.std(cl_noise_qubic, axis=0)[0]
    
    # The shape of cl_noise_qubic is : (#reals, #bands, #bins, 4)
    print('Old shape:', cl_noise_qubic.shape)
    cl_noise_qubic_reshape = np.moveaxis(cl_noise_qubic, [1, 2, 3], [3, 1, 2])
    print('New shape:', cl_noise_qubic_reshape.shape)
    # Covariance and correlation matrices for TT EE BB TE
    covbin, corrbin = amc.get_covcorr_patch(cl_noise_qubic_reshape, stokesjoint=True, doplot=False)


    return leff, mcl_noise_qubic, scl_noise_qubic, covbin
########################################################################################################


In [ ]:
### Run the MC
nbmc = 10
duration = 1.
leff, mcl_noise_qubic, scl_noise_qubic, covbin = run_mc(nbmc, Namaster, cov, d, configs, duration=duration)


In [ ]:
def ana_likelihood(rv, leff, fakedata, errors, model, prior, 
                   mylikelihood=mcmc.LogLikelihood, covariance_model_funct=None, otherp=None):
    ll = mylikelihood(xvals=leff, yvals=fakedata, errors=errors, 
                            model = model, flatprior=prior, covariance_model_funct=covariance_model_funct)  
    like = np.zeros_like(rv)
    for i in range(len(rv)):
        like[i] = np.exp(ll([rv[i]]))
    cumint = scipy.integrate.cumtrapz(like, x=rv)
    cumint = cumint / np.max(cumint)
    onesigma = np.interp(0.68, cumint, rv[1:])
    if otherp:
        other = np.interp(otherp, cumint, rv[1:])
        return like, cumint, onesigma, other
    else:
        return like, cumint, onesigma


def explore_like(leff, mcl_noise, errors, lmin, dl, cc, rv, otherp=None,
                 cov=None, plotlike=False, plotcls=False, 
                 verbose=False, sample_variance=True, mytitle='', color=None, mylabel='',my_ylim=None):
    
#     print(lmin, dl, cc)
#     print(leff)
#     print(scl_noise[:,2])
    ### Create Namaster Object
    # Unfortunately we need to recalculate fsky for calculating sample variance
    nside = 256
    lmax = 2 * nside - 1
    if cov is None:
        Namaster = nam.Namaster(None, lmin=lmin, lmax=lmax, delta_ell=dl)
        Namaster.fsky = 0.018
    else:
        okpix = cov > (np.max(cov) * float(cc))
        maskpix = np.zeros(12*nside**2)
        maskpix[okpix] = 1
        Namaster = nam.Namaster(maskpix, lmin=lmin, lmax=lmax, delta_ell=dl)
    
#     print('Fsky: {}'.format(Namaster.fsky))
    lbinned, b = Namaster.get_binning(nside)

    ### Bibnning CambLib
#     binned_camblib = qc.bin_camblib(Namaster, '../../scripts/QubicGeneralPaper2020/camblib.pickle', 
#                                     nside, verbose=False)
    binned_camblib = qc.bin_camblib(Namaster, global_dir + '/doc/CAMB/camblib.pkl', 
                                    nside, verbose=False)


    ### Redefine the function for getting binned Cls
    def myclth(ell,r):
        clth = qc.get_Dl_fromlib(ell, r, lib=binned_camblib, unlensed=False)[0]
        return clth
    allfakedata = myclth(leff, 0.)
    
    ### And we need a fast one for BB only as well
    def myBBth(ell, r):
        clBB = qc.get_Dl_fromlib(ell, r, lib=binned_camblib, unlensed=False, specindex=2)[0]
        return clBB

    ### Fake data
    fakedata = myBBth(leff, 0.)        
    
    if sample_variance:
        covariance_model_funct = Namaster.knox_covariance
    else:
        covariance_model_funct = None
    if otherp is None:
        like, cumint, allrlim = ana_likelihood(rv, leff, fakedata, 
                                            errors, 
                                            myBBth, [[0,1]],
                                           covariance_model_funct=covariance_model_funct)
    else:
        like, cumint, allrlim, other = ana_likelihood(rv, leff, fakedata, 
                                            errors, 
                                            myBBth, [[0,1]],
                                           covariance_model_funct=covariance_model_funct, otherp=otherp)
    
    if plotcls:
        if plotlike:
            subplot(1,2,1)
            if np.ndim(BBcov) == 2:
                errorstoplot = np.sqrt(np.diag(errors))
            else:
                errorstoplot = errors
        #plot(inputl, inputcl[:,2], 'k', label='r=0')
        plot(leff, errorstoplot, label=mylabel+' Errors', color=color)
        xlim(0,lmax)
        if my_ylim is None:
            ylim(1e-4,1e0)
        else:
            ylim(my_ylim[0], my_ylim[1])
        yscale('log')
        xlabel('$\\ell$')
        ylabel('$D_\\ell$')
        legend(loc='upper left')
    if plotlike:
        if plotcls:
            subplot(1,2,2)
        p=plot(rv, like/np.max(like), 
               label=mylabel+' $\sigma(r)={0:6.4f}$'.format(allrlim), color=color)
        plot(allrlim+np.zeros(2), [0,1.2], ':', color=p[0].get_color())
        xlabel('r')
        ylabel('posterior')
        legend(fontsize=8, loc='upper right')
        xlim(0,0.1)
        ylim(0,1.2)
        title(mytitle)
    
    if otherp is None:
        return like, cumint, allrlim
    else:
        return like, cumint, allrlim, other


In [ ]:
def plot_errors_lines(leff, err, dl, color='r', label=''):
    for i in range(len(leff)):
        if i==0:
            plot([leff[i]-dl/2, leff[i]+dl/2], [err[i,s], err[i,s]],color, label=label)
        else:
            plot([leff[i]-dl/2, leff[i]+dl/2], [err[i,s], err[i,s]],color)
        if i < (len(leff)-1):
            plot([leff[i]+dl/2,leff[i]+dl/2], [err[i,s], err[i+1,s]], color)
    

# camblib = qc.read_camblib(global_dir+'scripts/QubicGeneralPaper2020/camblib_TD.pickle')  

camblib = qc.read_camblib(global_dir + '/doc/CAMB/camblib.pkl')

rv = np.linspace(0,2,1000)

lll = np.arange(512)
cl0 = qc.get_Dl_fromlib(lll, 0, lib=camblib, unlensed=False)[0]   
cl0_01 = qc.get_Dl_fromlib(lll, 0.01, lib=camblib, unlensed=False)[0]   
cl0_06 = qc.get_Dl_fromlib(lll, 0.06, lib=camblib, unlensed=False)[0]   



In [ ]:
# Covariance matrix for BB, to use for the likelihood errors
BBcov = covbin[:, :, 2]
print(BBcov.shape)

scl = scl_noise_qubic.copy()
sclBB = scl[:, 2]
print(sclBB.shape)

subplot(2,1,1)
imshow(BBcov)
colorbar()
subplot(2,1,2)
imshow(qss.cov2corr(BBcov))
colorbar()



like, cumint, rlim68, rlim95 = explore_like(leff, scl*0, sclBB, lmin, delta_ell, covcut, rv,
                                 cov=cov, plotlike=False, plotcls=False, 
                                 verbose=True, sample_variance=True, otherp=0.95)


rc('figure', figsize=(15, 8))
rc('font', size=16)
clname = ['TT', 'EE', 'BB', 'TE']


figure()
for s in range(4):
    subplot(2,2,s+1)
    plot(lll, np.abs(cl0[:,s]),label='$\Lambda CDM$ r=0')
    if s==2:
        plot(lll, np.abs(cl0_01[:,s]),label='$\Lambda CDM$ r=0.01')
        plot(lll, np.abs(cl0_06[:,s]),label='$\Lambda CDM$ r=0.06')
    plot_errors_lines(leff, scl, delta_ell, 'r', label='QUBIC Noise')
    xlim(0,512)
    ylim(np.min([np.min(scl[:,s])/5,1e-4]) ,np.max([np.max(cl0[:512,s]*1.5),1.2*np.max(scl[:,s])]))
    yscale('log')
    xlabel('$\\ell$')
    ylabel('$D_\\ell$({})'.format(clname[s]))
    legend(loc='lower right')
    title('QUBIC {}x{} - {} years'.format(configs[0],configs[1], duration))
tight_layout()




In [ ]:
rc('figure', figsize=(15, 8))
rc('font', size=15)
subplot(1,2,1)
s=2
plot(lll, np.abs(cl0[:,s]),label='r=0')
if s==2:
    plot(lll, np.abs(cl0_01[:,s]),label='r=0.01')
    plot(lll, np.abs(cl0_06[:,s]),label='r=0.06')
plot_errors_lines(leff, scl, delta_ell, 'r', label='QUBIC {} - {} years'.format(d['config'], duration))
title(r'$\ell_m = {} \;-\; \Delta_\ell = {} \;-\; Cov_c = {}$'.format(lmin, delta_ell, covcut))
xlim(0,512)
ylim(np.min([np.min(scl[:,s])/5, 1e-4]),np.max([np.max(cl0[:512,s]*1.5),1.2*np.max(scl[:,s])]))
yscale('log')
xlabel('$\\ell$', fontsize=17)
ylabel('$D_\\ell$({})'.format(clname[s]), fontsize=17)
legend(loc='lower right')

subplot(1,2,2)
plot(rv,like, label='QUBIC {} - {} years'.format(d['config'], duration))
plot([rlim68, rlim68],[0,1.1],'k--', label='r < {0:5.3f} at 68% C.L.'.format(rlim68))
plot([rlim95, rlim95],[0,1.1],'k:', label='r < {0:5.3f} at 95% C.L.'.format(rlim95))
annotate("BKP-2018\n 95% C.L.", xy=(0.06, 0.01), xytext=(0.06, 0.2),
            arrowprops=dict(width=10, headwidth=20), ha='center')
title(r'$\ell_m = {} \;-\; \Delta_\ell = {} \;-\; Cov_c = {}$'.format(lmin, delta_ell, covcut))
legend()
xlabel('r')
ylabel('Posterior')
xlim(0,0.07)
ylim(0,1.1)